In [ ]:
!pip install openai
!pip install gradio

In [ ]:
import openai

OPENAI_API_KEY = ""
if not OPENAI_API_KEY:
    print("NEED OPENAI API Key from https://platform.openai.com/account/api-keys")
openai.api_key = OPENAI_API_KEY

In [ ]:
# OPENAI Example

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [ ]:
# OPENAI stream example

r = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
    ],
    stream=True
)

for chunk in r:
    print(chunk)

In [ ]:
# python method for one time query (display markdown output)

from IPython.display import Markdown

def chat_query(query, print_response=True):
    resp = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query},
        ]
    )
    
    if print_response:
        display(Markdown(r['choices'][0]['message']['content']))
    return resp

r = chat_query('write a python method to add 1 to 100')

In [ ]:
# Wrapper class

from IPython.display import Markdown, HTML
import time

class Chat:
    def __init__(self, system_content='You are a helpful assistant.'):
        self.system_content = system_content
        self.init_messages()
        
    def init_messages(self):
        self.messages = []
        if self.system_content:
            self.add_system_content(self.system_content)
            
    def add_user_content(self, content):
        self.messages.append({"role": "user", "content": content})
    def add_assistant_content(self, content):
        self.messages.append({"role": "assistant", "content": content})
    def add_system_content(self, content):
        self.messages.append({"role": "system", "content": content})
        
    def _run_query(self):
        max_retries = 7
        wait_time = 1

        for i in range(max_retries):
            try:
                self.response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=self.messages,
                    temperature=1.0,
                    top_p=1.0
                )
                return
            except Exception as e:
                if i == max_retries - 1:
                    raise
                else:
                    print(f"Exception {e}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                    wait_time *= 2

        
    def query(self, q, print_response=True):
        self.add_user_content(q)
        self._run_query()
        self.add_assistant_content(self.response['choices'][0]['message']['content'])
        if print_response:
            display(Markdown(self.messages[-1]['content']))
            
    def print_messages(self):
        for d in self.messages:
            role = d['role']
            content = d['content']
            if role == 'assistant':
                display(Markdown(content))
            else:
                color = '#080' if role == 'system' else '#008'
                display(HTML(f"<span style='color: {color}'><b>{role}</b> : {content}</span>"))
    


In [ ]:
chat = Chat()
chat.query('write a python exponential backoff code for openai.ChatCompletion.create')

In [ ]:
chat.query('write a python method to add 1 to 100')

In [ ]:
chat.query('change addition to multiplication')

In [ ]:
chat.print_messages()

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

chat = Chat()

def add_text(state, text):
    chat.query(text)
    result = chat.messages[-1]['content']
    state = state + [(text, result)]
    return state, state


with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    chatbot = gr.Chatbot(elem_id="chatbot")
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter, or upload an image").style(container=False)

    txt.submit(add_text, [state, txt], [state, chatbot])
    txt.submit(lambda :"", None, txt)

demo.launch(share=True, inline=True)


In [ ]:
demo.close()